# Undecided variables
Check whether, with no symmetry-breaking noise, the decided variables of a max-sum fixed point correspond to those in the optimal solution

In [1]:
include("../code/headers.jl")

In [34]:
bit2spin(x) = -2x+1
spin2bit(σ) = 1/2*(1-σ);

Build instance and duplicate it

In [33]:
lm1 = LossyModel(Val(2), 10000, 5000)
lm2 = deepcopy(lm1);

Solve with optimal cycle

In [28]:
solve!(lm1, OptimalCycle())
σ_optimal = bit2spin.(lm1.x);

Iter 1. Distortion 0.182. Cycle weight -6430.0. Energy shift -3215.0
Iter 2. Distortion 0.1416. Cycle weight -808.0. Energy shift -404.0
Iter 3. Distortion 0.1416. Cycle weight 0.0. Energy shift 0.0


Solve with Max-Sum

In [29]:
ms = MS(maxiter=1000, Tmax=1, convergence=:messages, tol=0.0, gamma=0.0, sigma=0.0)
@show solve!(lm2, ms)
σ_ms = sign.(lm2.fg.fields);

solve!(lm2, ms) = BPResults{MS}
  converged: Bool true
  parity: Int64 1548
  distortion: Float64 0.255
  trials: Int64 1
  iterations: Int64 8
  maxdiff: Array{Float64}((0,)) Float64[]
  codeword: BitArray{1}
  maxchange: Array{Float64}((1000,)) [2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]



Look only at decided variables and compare the two solutions

In [30]:
decided = σ_ms .!= 0
all(σ_optimal[decided]==σ_ms[decided])

false

In [42]:
lm1.x = zeros(Int, lm1.fg.n)
lm1.x[decided] = spin2bit.(σ_ms[decided])

for it = 1:10
    H = full_adjmat(lm1.fg)
    for v in 1:lm1.fg.n
        if lm1.x[v] != lm1.y[v]
            H[lm1.fg.m+v,:] .*= -1
            H[:,lm1.fg.m+v] .*= -1
        end      
    end
#     ms_decvars = guesses(lm2.fg)
    for v in findall(decided)
        H[lm1.fg.m+v,:] .*= -10000 
        H[:,lm1.fg.m+v] .*= -10000
    end
    
    op_, w = optimal_cycle(float.(H))
    op = variables_from_cycle(op_, lm1.fg.m)
    to_flip = unique!([tup[2] for tup in op])
    lm1.x[to_flip] .⊻= 1
    println(it, ". Distortion ", distortion(lm1))
end

1. Distortion 0.6003
2. Distortion 0.6932
3. Distortion 0.6909
4. Distortion 0.6908
5. Distortion 0.6908
6. Distortion 0.6908
7. Distortion 0.6908
8. Distortion 0.6908
9. Distortion 0.6908
10. Distortion 0.6908


In [31]:
sum(σ_optimal[decided] .!= σ_ms[decided])

156